In [2]:
import time, sys, string
import sqlite3
import json
import datetime
import calendar
import os
import fcntl
import re
import crcmod
from binascii import unhexlify
import paho.mqtt.client as mqtt
from random import randint

def connect():
    global client
    client = mqtt.Client(client_id=os.environ['MQTT_CLIENT_ID'])
    client.connect(os.environ['MQTT_SERVER'])
    try:
        global file
        global fd
        file = open('/dev/hidraw0', 'r+')
        fd = file.fileno()
        fl = fcntl.fcntl(fd, fcntl.F_GETFL)
        fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)
    except Exception as e:
        print('error open file descriptor: ' + str(e))
        exit()

def disconnect():
    file.close()

def serial_command(command):
    print(command)
    try:
        xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')
        command_crc = command + unhexlify(hex(xmodem_crc_func(command)).replace('0x','',1)) + '\x0d'

        os.write(fd, command_crc)

        response = ''
        timeout_counter = 0
        while '\r' not in response:
            if timeout_counter > 500:
                raise Exception('Read operation timed out')
            timeout_counter += 1
            try:
                response += os.read(fd, 100)
            except Exception as e:
                # print("error reading response...: " + str(e))
                time.sleep(0.01)
            if len(response) > 0 and response[0] != '(' or 'NAKss' in response:
                raise Exception('NAKss')

        print(response)
        response = response.rstrip()
        lastI = response.find('\r')
        response = response[1:lastI-2]
        return response
    except Exception as e:
        print('error reading inverter...: ' + str(e))
        disconnect()
        time.sleep(0.1)
        connect()
        return serial_command(command)

def get_parallel_data():
    #collect data from axpert inverter
    try:
        data = '{'
        response = serial_command('QPGS0')
        nums = response.split(' ')
        if len(nums) < 27:
            return ''

        if nums[2] == 'L':
            data += '"Gridmode":1'
        else:
            data += '"Gridmode":0'
        data += ',"SerialNumber": ' + str(int(nums[1]))
        data += ',"BatteryChargingCurrent": ' + str(int(nums[12]))
        data += ',"BatteryDischargeCurrent": ' + str(int(nums[26]))
        data += ',"TotalChargingCurrent": ' + str(int(nums[15]))
        data += ',"GridVoltage": ' + str(float(nums[4]))
        data += ',"GridFrequency": ' + str(float(nums[5]))
        data += ',"OutputVoltage": ' + str(float(nums[6]))
        data += ',"OutputFrequency": ' + str(float(nums[7]))
        data += ',"OutputAparentPower": ' + str(int(nums[8]))
        data += ',"OutputActivePower": ' + str(int(nums[9]))
        data += ',"LoadPercentage": ' + str(int(nums[10]))
        data += ',"BatteryVoltage": ' + str(float(nums[11]))
        data += ',"BatteryCapacity": ' + str(float(nums[13]))
        data += ',"PvInputVoltage": ' + str(float(nums[14]))
        data += ',"TotalAcOutputApparentPower": ' + str(int(nums[16]))
        data += ',"TotalAcOutputActivePower": ' + str(int(nums[17]))
        data += ',"TotalAcOutputPercentage": ' + str(int(nums[18]))
        # data += ',"InverterStatus": ' + nums[19]
        data += ',"OutputMode": ' + str(int(nums[20]))
        data += ',"ChargerSourcePriority": ' + str(int(nums[21]))
        data += ',"MaxChargeCurrent": ' + str(int(nums[22]))
        data += ',"MaxChargerRange": ' + str(int(nums[23]))
        data += ',"MaxAcChargerCurrent": ' + str(int(nums[24]))
        data += ',"PvInputCurrentForBattery": ' + str(int(nums[25]))
        if nums[2] == 'B':
            data += ',"Solarmode":1'
        else:
            data += ',"Solarmode":0'

        data += '}'
    except Exception as e:
        print('error parsing inverter data...: ' + str(e))
        return ''
    return data

def get_data():
    #collect data from axpert inverter
    try:
        response = serial_command('QPIGS')
        nums = response.split(' ')
        if len(nums) < 21:
            return ''

        data = '{'

        data += '"BusVoltage":' + str(float(nums[7]))
        data += ',"InverterHeatsinkTemperature":' + str(float(nums[11]))
        data += ',"BatteryVoltageFromScc":' + str(float(nums[14]))
        data += ',"PvInputCurrent":' + str(int(nums[12]))
        data += ',"PvInputVoltage":' + str(float(nums[13]))
        data += ',"PvInputPower":' + str(int(nums[19]))
        data += ',"BatteryChargingCurrent": ' + str(int(nums[9]))
        data += ',"BatteryDischargeCurrent":' + str(int(nums[15]))
        data += ',"DeviceStatus":"' + nums[16] + '"'

        data += '}'
        return data
    except Exception as e:
        print('error parsing inverter data...: ' + str(e))
        return ''

def send_data(data, topic):
    try:
        client.publish(topic, data)
    except Exception as e:
        print("error sending to emoncms...: " + str(e))
        return 0
    return 1

In [27]:
global file
global fd
file = open('/dev/hidraw1', 'r+')
fd = file.fileno()

In [89]:
command = 'QPIGS'

xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')
#command_crc = command + unhexlify(hex(xmodem_crc_func(command)).replace('0x','',1)) + '\x0d'
a=command.encode('utf-8')
b=unhexlify(hex(xmodem_crc_func(a)).replace('0x','',1))
c='\x0d'.encode('utf-8')
type(a+b+c)
command_crc=a+b+c

os.write(fd, command_crc)
response = ''
response = os.read(fd, 100)

#timeout_counter = 0
#while '\r' not in response:
#    if timeout_counter > 500:
#        raise Exception('Read operation timed out')
#    timeout_counter += 1
#    try:
#        response += os.read(fd, 100)
#    except Exception as e:
#        # print("error reading response...: " + str(e))
#        time.sleep(0.01)
#    if len(response) > 0 and response[0] != '(' or 'NAKss' in response:
#        raise Exception('NAKss')

print(response)

b'110P\x89\r\x00\x00'


In [102]:
# Moet response decode gebruik vir binary string
#print(response.decode("utf-8"))
print(response[0:4].find(b'\r'))
print(response[0:6])
disconnect()
# Dan gebruik die find funksie om die \r te kry en skryf die while lus hierbo oor.

-1
b'110P\x89\r'
